In [26]:
import requests
import json 
import os
import pandas as pd
import hvplot.pandas

In [27]:
#The api for tickers that the user will decide from
crypto_url = "https://api.coingecko.com/api/v3/exchanges/{0}/tickers?coin_ids={1}"
#list of markets places user can purchase crypto and cryptos available
top_five = ["gemini", "gdax", "kraken", "ftx_spot"]
crypto_list = ["ethereum", "bitcoin", "dogecoin", "matic-network"]


In [28]:
# running a loop to find each exchange that trades the coins listed in crypto_list to USD
#dataframe is an empty variable with columns
def coin(crypto):
    dataframe = pd.DataFrame(columns = ["Timestamp", "Market", "Base", "Last"])
    for exchange in top_five:
        platform = requests.get(crypto_url.format(exchange, crypto)).json()
        for currency in platform["tickers"]:
            if currency["target"] == "USD":
                #print(json.dumps(currency,indent = 4, sort_keys = True))
                time = pd.Timestamp(currency["timestamp"], tz = "America/Chicago")
                # set the colmumns (keys from the dictionary) to match the of each coin and exchange values 
                dataframe.loc[len(dataframe.index)] = [time, currency["market"]["name"], currency["base"], currency["last"]]
    return dataframe

In [29]:
# This will print out the dictionary for the Coin-USD 
ticker = crypto_list[3]
dataframe = coin(crypto_list[3])
minimum = dataframe["Last"].min() - (dataframe["Last"].min() * 0.0025)
maximum = dataframe["Last"].max() + (dataframe["Last"].max() * 0.0025)

dataframe.hvplot.bar( title = f"Arbitrage for Exchanges {ticker}", 
                   figsize = (20,15),
                   x = "Market",
                   y = "Last",
                     ylabel = "Coin Price",
                     xlabel = "Exchange",
                     ylim = (minimum, maximum)).opts( yformatter = "%.6f")

:Bars   [Market]   (Last)